In [2]:
import pandas as pd

## Preparing the Data for Cancer Sub-Type Classification

In [3]:
df_lung = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/lung.csv')
df_normal = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/normal.csv')

In [4]:
def create_combined_df(cancer_df, normal_df, random_state):
    cancer_count = len(cancer_df)
    normal_sample = normal_df.sample(n=cancer_count, random_state=random_state, replace=False)

    combined_df = pd.concat([cancer_df, normal_sample], ignore_index=True)
    return combined_df

In [5]:
df_lung_cancer = create_combined_df(df_lung, df_normal, 8)
df_lung_cancer.shape

(230, 54677)

In [6]:
df_lung_cancer['type'].unique()

array(['tumoral', 'non_small_cell_lung_cancer', 'normal'], dtype=object)

In [7]:
def preprocessing_sub(df):
    
    # Drop cancer_type Columns
    if "cancer_type" in df.columns:
        df = df.drop(columns="cancer_type")

    # Get X,y
    target = 'type'
    X = df.drop(columns=target)
    y = df[target]
    
    return X,y

# Display proccesed data

Xs_l,ys_l = preprocessing_sub(df_lung_cancer)

display(Xs_l,ys_l)

,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,12.014762,6.983442,6.540233,8.362803,3.780203,9.188556,5.674197,6.262662,9.220699,4.121583,...,13.391406,12.688067,14.222519,14.145409,10.784380,9.229371,10.030957,3.569567,4.249465,3.982578
1,11.317501,7.243950,6.927529,8.374879,3.845977,8.546901,5.680735,5.811109,8.687212,4.578750,...,13.307399,12.579227,14.192594,14.145906,9.772229,7.742450,8.501174,3.532915,4.134520,3.917711
2,10.868398,7.213200,7.110826,8.258420,4.074300,9.295490,5.494098,5.788352,10.384183,4.558008,...,13.747090,13.054736,14.391262,14.313572,11.928554,10.481570,11.281070,3.453530,4.226954,3.706964
3,11.968264,8.003929,7.167021,8.794291,3.679181,8.404464,5.877476,5.746267,8.973831,4.575594,...,12.679800,11.850551,13.872564,13.703017,4.679738,4.777636,4.329992,3.568955,4.091501,3.881763
4,11.048648,7.688655,8.113348,8.656912,3.542036,8.778009,5.740964,5.467730,10.060357,4.469260,...,13.114882,12.372716,14.019266,13.810847,4.575873,4.715638,4.442247,3.573208,3.906953,3.848166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,10.537254,7.459961,7.743787,8.062182,3.735166,8.963168,5.650002,6.039367,9.768300,4.189641,...,13.001646,12.227844,14.057034,13.984470,12.149398,10.538870,11.258644,3.815355,4.191686,3.913278
226,10.951526,6.032387,5.513643,8.808598,3.677768,8.913564,6.449267,4.852616,8.049835,3.671817,...,12.251976,11.624545,13.554286,13.401902,9.576175,5.123975,6.967157,3.287151,3.802152,3.592062
227,10.477503,7.827490,8.480160,8.595126,3.630788,8.676451,6.263245,6.034697,11.033220,4.494361,...,12.926594,12.060730,13.953663,13.810157,12.649718,6.987827,10.621531,3.871506,4.074219,4.006980
228,10.465742,7.757333,9.566909,8.579894,3.673610,8.562270,5.447044,6.850716,10.761728,4.289232,...,12.468944,11.452733,13.696956,13.636120,8.787804,6.011892,7.236161,3.574150,3.953565,3.978115


0      tumoral
1      tumoral
2      tumoral
3      tumoral
4      tumoral
        ...   
225     normal
226     normal
227     normal
228     normal
229     normal
Name: type, Length: 230, dtype: object

## Sub-Type Classification using Logistic Regression

In [12]:
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def kfold_with_pca(X, y, n_components, n_splits=5):
    
    # Ensure consistent column ordering
    feature_names = X.columns.tolist()

    # Get the unique class labels
    unique_classes = np.unique(y)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    pca = PCA(n_components=n_components)
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Lists to store fold metrics
    accuracies, recalls, precisions, f1_scores = [], [], [], []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Apply PCA
        X_train_pca = pca.fit_transform(X_train)
        X_test_pca = pca.transform(X_test)

        # Train the model
        model.fit(X_train_pca, y_train)

        # Predict on the test set
        y_pred = model.predict(X_test_pca)

        # Calculate metrics
        accuracies.append(accuracy_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        precisions.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        f1_scores.append(f1_score(y_test, y_pred, average='macro'))

    # Final PCA fitting and model training on full data
    X_pca_full = pca.fit_transform(X)
    model.fit(X_pca_full, y)

    # Aggregate metrics
    metrics = {
        "Mean Accuracy": np.mean(accuracies),
        "Mean Recall": np.mean(recalls),
        "Mean Precision": np.mean(precisions),
        "Mean F1 Score": np.mean(f1_scores),
    }

    # Print cross-validation results
    print(f"5-Fold CV Results with {n_components} PCA components:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.2f}")

    # Return the final model, PCA object, and metrics
    return {
        "model": model,
        "pca": pca,
        "feature_names": feature_names,
        "metrics": metrics,
    }


In [14]:
results = kfold_with_pca(Xs_l, ys_l, n_components=50)
results


5-Fold CV Results with 50 PCA components:
Mean Accuracy: 0.96
Mean Recall: 0.95
Mean Precision: 0.97
Mean F1 Score: 0.95


{'model': RandomForestClassifier(random_state=42),
 'pca': PCA(n_components=50),
 'feature_names': ['1007_s_at',
  '1053_at',
  '117_at',
  '121_at',
  '1255_g_at',
  '1294_at',
  '1316_at',
  '1320_at',
  '1405_i_at',
  '1431_at',
  '1438_at',
  '1487_at',
  '1494_f_at',
  '1552256_a_at',
  '1552257_a_at',
  '1552258_at',
  '1552261_at',
  '1552263_at',
  '1552264_a_at',
  '1552266_at',
  '1552269_at',
  '1552271_at',
  '1552272_a_at',
  '1552274_at',
  '1552275_s_at',
  '1552276_a_at',
  '1552277_a_at',
  '1552278_a_at',
  '1552279_a_at',
  '1552280_at',
  '1552281_at',
  '1552283_s_at',
  '1552286_at',
  '1552287_s_at',
  '1552288_at',
  '1552289_a_at',
  '1552291_at',
  '1552293_at',
  '1552295_a_at',
  '1552296_at',
  '1552299_at',
  '1552301_a_at',
  '1552302_at',
  '1552303_a_at',
  '1552304_at',
  '1552306_at',
  '1552307_a_at',
  '1552309_a_at',
  '1552310_at',
  '1552311_a_at',
  '1552312_a_at',
  '1552314_a_at',
  '1552315_at',
  '1552316_a_at',
  '1552318_at',
  '1552319_a_

In [15]:
model = results["model"]
pca = results["pca"]

In [16]:
df_test = pd.read_csv('/Users/ledamduyen/Desktop/CS 539/project/dataset/clean/test_data.csv')
df_lung_test = df_test[df_test['cancer_type'].isin(['normal', 'lung'])]

In [17]:
X_test,y_test = preprocessing_sub(df_lung_test)

In [18]:
from sklearn.metrics import classification_report

def evaluate_model(model, pca, X_test, y_test, unique_classes, feature_names):
    """
    Evaluate the final trained model on test data.
    """
    # Ensure test set columns match the training set
    X_test = X_test[feature_names]

    # Transform test data using PCA
    X_test_pca = pca.transform(X_test)

    # Predict using the final trained model
    y_pred = model.predict(X_test_pca)

    # Calculate overall accuracy
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average=None, labels=unique_classes, zero_division=0)

    # Print results
    print("\nFinal Model Evaluation on Test Dataset:")
    print(f"Accuracy: {accuracy:.2f}")
    print("\nRecall for each class:")
    for i, cls in enumerate(unique_classes):
        print(f"Class {cls}: Recall = {recall[i]:.2f}")

    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, labels=unique_classes))


In [ ]:
evaluate_model(
    model=results["model"],
    pca=results["pca"],
    X_test=X_test,
    y_test=y_test,
    unique_classes=np.unique(ys_l),
    feature_names=results["feature_names"],
)

5-Fold CV Results with 50 PCA components:
Mean Accuracy: 0.95
Mean Recall: 0.93
Mean Precision: 0.97
Mean F1 Score: 0.95

Final Model Evaluation on Test Dataset:
Accuracy: 1.00

Recall for each class:
Class non_small_cell_lung_cancer: Recall = 1.00
Class normal: Recall = 1.00
Class tumoral: Recall = 1.00

Classification Report:
                            precision    recall  f1-score   support

non_small_cell_lung_cancer       1.00      1.00      1.00         4
                    normal       1.00      1.00      1.00        52
                   tumoral       1.00      1.00      1.00         7

                  accuracy                           1.00        63
                 macro avg       1.00      1.00      1.00        63
              weighted avg       1.00      1.00      1.00        63



In [20]:
# Save model

import joblib
joblib.dump(model, 'lung_sub.joblib')
joblib.dump(pca, 'lung_sub_pca.joblib')

['lung_sub_pca.joblib']